# First Query to Gaia Data

In this notebook I will create the first connection to Gaia Data, execute some ADQL query, download data, perform some basic analysis and create some visualization. 
The purpose is just test the connection and get familiar with ADQL and the process to get data from Gaia.

### 1. Imports and configs

In [ ]:
# Import libraries

import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from astroquery.gaia import Gaia 
from astropy import units as u
from astropy.coordinates import SkyCoord
import getpass
from IPython.display import display
import ipywidgets as widgets
import warnings
warnings.filterwarnings("ignore")

In [ ]:
plt.style.use('default')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

### 2. Login to Gaia Service

I will create this code to login to Gaia Service, using ipywidgets to ask for user/pass. I also created the login to connect anonymously in case user/pass are not provided or incorrect

In [ ]:
Gaia.MAIN_GAIA_TABLE == 'gaiadr3.gaia_source' #use the DR3 catalog
login_checkbox = True  # Change to False if you want an anonymous login
use_login = login_checkbox

print("\\n--- INTERACTIVE CONFIGURATION ---")
login_checkbox = widgets.Checkbox(value=True, description='Login to ESA archive?')
username_text = widgets.Text(placeholder='User', description='Enter your user name to connect to ESA archive')
password_text = widgets.Password(placeholder='Password', description='Password')
login_button = widgets.Button(description='Connect')

def on_login_click(b):
    global USE_LOGIN, USERNAME, PASSWORD
    USE_LOGIN = login_checkbox
    USERNAME = username_text.value
    PASSWORD = password_text.value
    print("Credentials saved, run the following cell to connect")

login_button.on_click(on_login_click)

display(login_checkbox)
if login_checkbox.value:
    display(username_text)
    display(password_text)
    display(login_button)
print("=== CONFIG OF ACCESS TO GAIA DATA ===")
print("\nAccess options:")
print("1. Anonymous (limits: ~3GB queries, 2000 rows sync)")
print("2. Registered access (recomended: bigger queries)")

if use_login =='True':
    print("\n--- LOGIN WITH ESA USER ---")
    print("If you don't have an account, register in: https://www.cosmos.esa.int/web/gaia-users/register")
    
    try:
        # Attempt to log in
        print(f"Trying to connect with: {USERNAME}")
        Gaia.login(user=USERNAME, password=PASSWORD)
        print("✅ Successful login!")
        
        # Check login status
        # print(f"✅ Conected as: {Gaia.get_user()}")
        
        # With a registered user, we can use higher limits.
        Gaia.ROW_LIMIT = 2000  # Higher limit for registered users
        
        # Display account information
        print("\n--- ACCOUNT INFORMATION ---")
        try:
            print("Checking account limits...")
            # Note: the API may not always return this information.
        except:
            print("Quota information not available via API")
    except Exception as e:
        print(f"❌ Login error: {e}")
        print("Continuing with anonymous access...")
        print("Please verify your credentials at: https://gea.esac.esa.int/archive/")
        Gaia.ROW_LIMIT = 50  # Lower limit for anonymous access
        USE_LOGIN = False
else:
    print("Continuing with anonymous access...")
    Gaia.ROW_LIMIT = 50  # Lower limit for anonymous access

print(f"\nFinal configuration:")
print(f"Main table: {Gaia.MAIN_GAIA_TABLE}")
print(f"Row limit per query: {Gaia.ROW_LIMIT}")


### 3. Bright stars within 50 parsecs

##### 3.1. ADQL query to obtain bright stars within 50 parsecs

In [ ]:
# build ADQL query taking some fields (coordinates, parallax, magnitude, bp_rp, radial velocity)
query1 = f"""
SELECT TOP {Gaia.ROW_LIMIT}
    source_id,
    ra, dec,
    parallax,
    parallax_error,
    phot_g_mean_mag,
    bp_rp,
    radial_velocity,
    pmra, pmdec
FROM gaiadr3.gaia_source
WHERE parallax > 20
    AND parallax_over_error > 5
    AND phot_g_mean_mag IS NOT NULL
ORDER BY phot_g_mean_mag ASC
"""

print(f"Consulting the {Gaia.ROW_LIMIT} brightest stars nearby...")

# Run the query synchronously
job1 = Gaia.launch_job(query1)
results1 = job1.get_results()

print(f"Query completed. Obtained {len(results1)} stars.")
print("\nFirst 5 stars:")
print(results1[:5].to_pandas())

##### 3.2. Exploratory Data Analysis

In [ ]:
print("\n=== EXPLORATORY DATA ANALYSIS ===")

# Convert to Pandas DataFrame for easier analysis
df = results1.to_pandas()

# Calculate distance in parsecs (1/parallax)
df['distance_pc'] = 1000 / df['parallax']  # parallax in mas -> distance in pc

print(f"Brightest magnitude: {df['phot_g_mean_mag'].min():.2f}")
print(f"Closest distance: {df['distance_pc'].min():.1f} parsecs")
print(f"Average B-R colour: {df['bp_rp'].mean():.2f}")

##### 3.3. Visualization

In [ ]:
# Create a 2*2 matrix to display 4 diagrams
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

# Fig 1: Colour-magnitude diagram
ax1.scatter(df['bp_rp'], df['phot_g_mean_mag'], alpha=0.7, s=50)
ax1.set_xlabel('Colour (BP-RP)')
ax1.set_ylabel('G Magnitude')
ax1.set_title('Colour-magnitude diagram')
ax1.invert_yaxis()  # Brighter magnitudes above
ax1.grid(True, alpha=0.3)

# Figure 2: Histogram of distances
ax2.hist(df['distance_pc'], bins=10, alpha=0.7, edgecolor='black')
ax2.set_xlabel('Distance (parsecs)')
ax2.set_ylabel('Num of stars')
ax2.set_title('Distribution of Distances')
ax2.grid(True, alpha=0.3)

# Figure 3: Distance-magnitude relationship
ax3.scatter(df['distance_pc'], df['phot_g_mean_mag'], alpha=0.7, s=50)
ax3.set_xlabel('Distance (parsecs)')
ax3.set_ylabel('G Magnitude')
ax3.set_title('Distance vs Magnitude')
ax3.grid(True, alpha=0.3)

# Figure 4: Distribution in the sky (RA, Dec)
scatter = ax4.scatter(df['ra'], df['dec'], c=df['phot_g_mean_mag'], 
                     cmap='viridis_r', s=50, alpha=0.7)
ax4.set_xlabel('Right Asc (degrees)')
ax4.set_ylabel('Declination (degrees)')
ax4.set_title('Position in the sky')
ax4.grid(True, alpha=0.3)
plt.colorbar(scatter, ax=ax4, label='G Magnitude')

plt.tight_layout()
plt.show()